In [47]:
import pandas as pd
import joblib
from sklearn.metrics import confusion_matrix
import json
import pickle
import numpy as np

In [48]:
lg1 = joblib.load('/home/lxu/drugrelink/resources/predictive_model/edge2vec/0/logistic_regression.joblib')

In [49]:
with open('/home/lxu/drugrelink/resources/predictive_model/edge2vec/0/word2vec_model.pickle','rb') as f:
    wv = pickle.load(f)

In [50]:
with open('repurpose_overlap.json') as f:
    repurpose = json.load(f)

In [51]:
repo = pd.read_csv('repo_data.csv',index_col=False)


In [52]:
repo.head()
repo_pos = repo.loc[repo['status']==1]
repo_neg = repo.loc[repo['status']==0]
pos_list=list(map(tuple, repo_pos.values))
pos = np.array(pos_list)[:,1:3]
pos_label = np.array(pos_list)[:,3]
neg_list = list(map(tuple, repo_neg.values))
neg = np.array(neg_list)[:,1:3]
neg_label = np.array(neg_list)[:,3]

In [53]:
pl = []
nl = []
for i in pos_label:
    i = float(i)
    pl.append(i)
for j in neg_label:
    j = float(j)
    nl.append(j)

In [54]:
from drugrelink.embedders import get_embedder

In [55]:
embedder_function = get_embedder('hardamard')
vectors = np.array(embedder_function(wv, repurpose))
labels = np.array([1]*len(repurpose))

In [56]:
pos_repo_vec = np.array(embedder_function(wv, pos))
neg_repo_vec = np.array(embedder_function(wv, neg))
a =lg1.score(pos_repo_vec,pl)
b = lg1.score(neg_repo_vec,nl)
print(a,b)

0.8463414634146341 1.0


In [57]:
a =lg1.score(vectors,labels)
print(a)

0.8536585365853658


In [9]:
lg1.get_params

<bound method BaseEstimator.get_params of LogitNet(alpha=0.2, cut_point=1.0, fit_intercept=True, lambda_path=None,
     lower_limits=-inf, max_features=None, max_iter=100000,
     min_lambda_ratio=1e-08, n_jobs=10, n_lambda=150, n_splits=3,
     random_state=2, scoring=None, standardize=True, tol=1e-07,
     upper_limits=inf, verbose=False)>

In [10]:
lg1.coef_

array([[ 0.00024994,  0.        ,  0.02188158,  0.        ,  0.01342754,
         0.        ,  0.01057434,  0.        ,  0.        ,  0.00239645,
         0.00761076,  0.00212515,  0.00044114, -0.00811494,  0.        ,
         0.0064581 ,  0.0074401 ,  0.00802477,  0.        ,  0.00892437,
         0.0298366 ,  0.01006643,  0.0168908 ,  0.00184513,  0.00649053,
         0.00133396,  0.00841055,  0.        ,  0.01223701,  0.02492136,
         0.01177773,  0.00956241,  0.02491351,  0.        ,  0.        ,
         0.00702691,  0.00557689, -0.01060999,  0.        , -0.00055949,
         0.01780001,  0.        ,  0.00857241,  0.        ,  0.        ,
         0.01544074,  0.        ,  0.        ]])

In [11]:
lg1.cv_mean_score_

array([0.56498906, 0.59766594, 0.59941648, 0.6079504 , 0.61983953,
       0.63245806, 0.6460248 , 0.65244347, 0.6547046 , 0.65660102,
       0.66250912, 0.66827133, 0.6730124 , 0.67760759, 0.68278629,
       0.68825675, 0.69102845, 0.69533187, 0.69737418, 0.70021882,
       0.70226112, 0.70255288, 0.70386579, 0.70503282, 0.70671043,
       0.70707513, 0.70714807, 0.70773158, 0.70882567, 0.7086798 ,
       0.70889861, 0.70838804, 0.70875274, 0.70911743, 0.71043034,
       0.71064916, 0.71086798, 0.71094092, 0.71137856, 0.71130562,
       0.71152443, 0.71196207, 0.71225383, 0.71203501, 0.71188913,
       0.71188913, 0.71188913, 0.71196207, 0.71188913, 0.71239971,
       0.71210795, 0.71188913, 0.71137856, 0.71123268, 0.71130562,
       0.71137856, 0.7114515 , 0.71152443, 0.7114515 , 0.71137856,
       0.71130562, 0.71130562, 0.71137856, 0.71130562, 0.71115974,
       0.71115974])

In [12]:
lg1.n_lambda_

66

In [13]:
repo_data = pd.read_csv('repo_data.csv')

In [45]:
from drugrelink.pairs import data_non_overlap, pairs_vectors

In [ ]:
data_non_overlap(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv', train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [60]:
from drugrelink.embedders import get_embedder
import os
def reevaluate (path):
    s=[]
    for i,name in enumerate(os.listdir(path), start=1):
        print(name)
        if name in ['1','2','3','4','5','6','7','8','9','10']:
            subpath = os.path.join(path,name)
            print(subpath)
            lg_path = os.path.join(subpath,'logistic_regression_clf.joblib')
            lg1 = joblib.load(lg_path)
            wv_path = os.path.join(subpath,'word2vec_model.pickle') 
            with open(wv_path,'rb') as f:
                wv = pickle.load(f)
            embedder_function = get_embedder('hardamard')
            vectors = np.array(embedder_function(wv, repurpose))
            labels = np.array([1]*len(repurpose))
            a=lg1.score(vectors,labels)
            print(a)
            s.append(a)
            print(s)
        else:
            continue
    return s

In [61]:
s1 = reevaluate('/home/lxu/Downloads/node2vec_graph_d128')

config.json
9
/home/lxu/Downloads/node2vec_graph_d128/9
0.04878048780487805
[array(0.04878049)]
8
/home/lxu/Downloads/node2vec_graph_d128/8
0.024390243902439025
[array(0.04878049), array(0.02439024)]
run.sh
7
/home/lxu/Downloads/node2vec_graph_d128/7
0.0
[array(0.04878049), array(0.02439024), array(0.)]
4
/home/lxu/Downloads/node2vec_graph_d128/4
0.04878048780487805
[array(0.04878049), array(0.02439024), array(0.), array(0.04878049)]
1
/home/lxu/Downloads/node2vec_graph_d128/1
0.024390243902439025
[array(0.04878049), array(0.02439024), array(0.), array(0.04878049), array(0.02439024)]
slurm-232741.out
5
/home/lxu/Downloads/node2vec_graph_d128/5
0.04878048780487805
[array(0.04878049), array(0.02439024), array(0.), array(0.04878049), array(0.02439024), array(0.04878049)]
3
/home/lxu/Downloads/node2vec_graph_d128/3
0.024390243902439025
[array(0.04878049), array(0.02439024), array(0.), array(0.04878049), array(0.02439024), array(0.04878049), array(0.02439024)]
metadata.json
2
/home/lxu/Down

In [62]:
print(s1)

[array(0.04878049), array(0.02439024), array(0.), array(0.04878049), array(0.02439024), array(0.04878049), array(0.02439024), array(0.02439024), array(0.02439024), array(0.04878049)]


In [64]:
s2 = reevaluate('/home/lxu/Desktop/edge2vec_em10')

slurm-195308.out
9
/home/lxu/Desktop/edge2vec_em10/9
0.0
[array(0.)]
run.sh
slurm-195361.out
10
/home/lxu/Desktop/edge2vec_em10/10
0.0
[array(0.), array(0.)]
5
/home/lxu/Desktop/edge2vec_em10/5
0.0
[array(0.), array(0.), array(0.)]
8
/home/lxu/Desktop/edge2vec_em10/8
0.0
[array(0.), array(0.), array(0.), array(0.)]
2
/home/lxu/Desktop/edge2vec_em10/2
0.0
[array(0.), array(0.), array(0.), array(0.), array(0.)]
3
/home/lxu/Desktop/edge2vec_em10/3
0.024390243902439025
[array(0.), array(0.), array(0.), array(0.), array(0.), array(0.02439024)]
7
/home/lxu/Desktop/edge2vec_em10/7
0.024390243902439025
[array(0.), array(0.), array(0.), array(0.), array(0.), array(0.02439024), array(0.02439024)]
slurm-195250.out
6
/home/lxu/Desktop/edge2vec_em10/6
0.0
[array(0.), array(0.), array(0.), array(0.), array(0.), array(0.02439024), array(0.02439024), array(0.)]
config.json
1
/home/lxu/Desktop/edge2vec_em10/1
0.0
[array(0.), array(0.), array(0.), array(0.), array(0.), array(0.02439024), array(0.0243902

In [74]:
wv = pd.read_csv('/home/lxu/drugrelink/resources/predictive_model/edge2vec/0/word2vec_wv')

In [73]:
wv.shape

(45158, 1)

In [78]:
wv.columns = ['name']

In [79]:
wv.head()

,name
0,Anatomy::UBERON:0000473 0.61479926 -1.5700234 ...
1,Anatomy::UBERON:0000955 2.8624015 -2.018733 0....
2,Anatomy::UBERON:0002369 1.5198268 -1.2577668 1...
3,Anatomy::UBERON:0002048 0.9906945 -1.2947552 3...
4,Anatomy::UBERON:0002368 7.8613386 -0.038186327...


In [80]:
for _, row in wv.iterrows():
    if 'Compound::DB01377' in row['name']:
        print('1')